athena2pyspark
==

Usted está frente athena2pyspark una api creada por exalítica para la manipulación de consultas en sql desde s3 basado en athena. El objetivo de esta librería es no tener un futuro sin servidores y sin cluster EMR. Cuide esta librería, apoye reportando bugs y disfrútela.

Instalación
==
pip install git+https://ljofre-exalitica@bitbucket.org/exalitica-team/athena2pyspark.git

In [1]:
import athena2pyspark as ath # se demora un poco porque esta creado el SparkContext

In [2]:
from athena2pyspark.athena_sql.dinamicas import producto_nuevo

por ejemplo, tenemos la función que retorna la consulta de producto nuevo

In [3]:
producto_nuevo_query = producto_nuevo(subclase=110209, marca='2717', lift=8)
print(producto_nuevo_query)


    WITH fuente_filtrada AS
    (
    with items_filtrado AS (
        SELECT DISTINCT item_hash.item_id
        FROM prod_jumbo.item_hash
        WHERE item_hash.item_subclass_cd = 110209
                OR item_hash.brand_cd = '2717')
    SELECT fuente.party_id, fuente.tran_start_dt, fuente.item_id from fuente join items_filtrado
    on fuente.item_id = items_filtrado.item_id
      where cast(fuente.party_id as integer) > 0
    )
    
    -- select count(*) from fuente_filtrada 35 929 589
    
    ,
    subclasepop AS
    (
      SELECT fuente_filtrada.party_id,
             item_hash.item_subclass_cd,
             COUNT(DISTINCT (fuente_filtrada.tran_start_dt)) AS visitas
      FROM fuente_filtrada
        LEFT JOIN item_hash ON (fuente_filtrada.item_id = item_hash.item_id)
      GROUP BY fuente_filtrada.party_id,
               item_hash.item_subclass_cd
    )
    
    -- select count(*) from subclasepop 15867270
    
    ,
    marcapop AS
    (
      SELECT fuente_filtrada.party_

Hacer una query y que te retorne un dataframe
==

Podemos usar los recursos de athena para obtener querys basado en sql, esto nos permite hacer querys gigantes de forma server-less

In [4]:
s3_output = "s3://leonardo.exalitica.com/boto3/query_1/"
path_dataframe = ath.run_query(query = "select * from baul_2 limit 10", database = "ljofre", s3_output = s3_output)

Execution ID: 8998c755-3943-4ffa-8bb9-d2211ce279bf


In [6]:
df = ath.get_dataframe(path_query=path_dataframe)
df.show()

+---------+-----+-----+------+-------+----+---------+------------------+-----+------+-------+----+----+-----+-------+
| party_id| corr|cv_us|cv_u4s|cv_u12s|m_us|    m_u4s|            m_u12s|cp_us|cp_u4s|cp_u12s| rec|tu4s|tu12s|  n_key|
+---------+-----+-----+------+-------+----+---------+------------------+-----+------+-------+----+----+-----+-------+
|117989852|23353|    0|     0|      1|   0|      0.0|         2368.9076|    0|   0.0|    1.0|86.0|  30| 90.0|2016_10|
|113712744|12817|    0|     0|      1|   0|      0.0|         2008.4034|    0|   0.0|    1.0|64.0|  30| 90.0|2016_10|
|118189345| 7742|    0|     1|      2|   0| 838.6554|         2063.8654|    0|   3.0|    5.0|24.0|  30| 65.0|2016_10|
|113901510|27814|    0|     0|      1|   0|      0.0|         2267.2269|    0|   0.0|   0.45|52.0|  30| 90.0|2016_10|
|118386819|19692|    0|     0|      3|   0|      0.0|13537.815200000001|    0|   0.0|    9.0|45.0|  30| 10.5|2016_10|
|114096817|   37|    0|     0|      1|   0|      0.0|   

Dinámica producto nuevo
==

Podemos obtener el dataframe de dinámica de producto nuevo

In [7]:
path_producto_nuevo = ath.run_query(query = producto_nuevo_query, database = "prod_jumbo", s3_output = s3_output)

Execution ID: 84aa9c03-36cb-4700-b02f-edca1bf7b3ab


In [8]:
df_producto_nuevo = ath.get_dataframe(path_query=path_producto_nuevo)
df_producto_nuevo.show()

+-----+-----+
|_col0|_col1|
+-----+-----+
|12094|  600|
+-----+-----+



Generar el "create table" a partir del dataframe ya creado
==

Es util registrar este dataframe dentro de un catálogo para poder seguir haciendo consultas dentro de athena

In [8]:
s3_input = "s3://leonardo.exalitica.com/boto3/query_1/8da33860-e60e-42a9-9e83-79120a4a7c95.csv"
create_database, create_table = ath.get_ddl(df=df,
                                            database="ljofre",
                                            table="nueva_tabla",
                                            s3_input=s3_input)

In [11]:
print(create_database)

CREATE DATABASE IF NOT EXISTS ljofre;


In [12]:
print(create_table)

CREATE EXTERNAL TABLE IF NOT EXISTS ljofre.nueva_tabla ('party_id' string,'corr' string,'cv_us' string,'cv_u4s' string,'cv_u12s' string,'m_us' string,'m_u4s' string,'m_u12s' string,'cp_us' string,'cp_u4s' string,'cp_u12s' string,'rec' string,'tu4s' string,'tu12s' string,'n_key' string)
     ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
     WITH SERDEPROPERTIES (
     'serialization.format' = '1'
     ) LOCATION 's3://leonardo.exalitica.com/boto3/query_1/8da33860-e60e-42a9-9e83-79120a4a7c95.csv'
     TBLPROPERTIES ('has_encrypted_data'='false');


Ejecutar create table y dejar registrada la tabla en Athena
==

In [15]:
import boto3
client = boto3.client('athena', region_name='us-east-1', 
                      aws_access_key_id=aws_access_key_id,
                      aws_secret_access_key=aws_secret_access_key)

response = client.start_query_execution(
    QueryString=create_table,
    QueryExecutionContext={
        'Database': database
        }
    )
print('Execution ID: ' + response['QueryExecutionId'])

NameError: name 'aws_access_key_id' is not defined

Hacer un listado a partir de la prepriorizacion
==